<img src="https://github.com/fjmeyer/HydroSAR/raw/master/HydroSARbanner.jpg" width="100%" />

### <font size="7"> <b> Download HAND from GEE</b></font>

<font size="5">  Download raster tiles from HAND dataset on Google Earth Engine </font>

<br>
<font size="4"> <b> Part of NASA A.37 Project:</b> Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters   <br>
<font size="4"> <b> PI:</b>Franz J. Meyer <br>
<font size="3"> Version 0.1.0 - 2021/01/13 <br>
<b>Change Log</b><br>
See bottom of the notebook.<br>
</font> 
<font color='rgba(0,0,200,0.2)'> <b>Contact: </b> batuhan.osmanoglu@nasa.gov </font>


<hr>
<font face="Calibri">

<font size="5"> <b> 0. Importing Relevant Python Packages </b> </font>

<font size="3"> The first step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following libraries:
<ol type="1">
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li> 
    <li> <b><a href="https://docs.python.org/3/library/urllib.html" target="_blank">urllib</a></b> is an internal package that collects several modules for working with URLs.</li>
    <li> <b><a href="https://docs.python.org/3/library/zipfile.html" target="_blank">zipfile</a></b> is an internal python module provides tools to create, read, write, append, and list a ZIP file.</li>
    <li> <b><a href="https://docs.python.org/3/library/tempfile.html" target="_blank">tempfile</a></b> is an internal python package that creates temporary files and directories.</li>
    <li> <b><a href="https://github.com/tqdm/tqdm" target="_blank"> tqdm </a></b> is a smart progress meter that allows easy addition of a loop counter.</li>
    <li> <b><a href="https://pypi.org/project/google-api-python-client/" target="_blank">googleapiclient</a></b> is the Python client library for Google's discovery based APIs. These client libraries are officially supported by Google. </li>
    <li> <b><a href="https://pypi.org/project/oauth2client/" target="_blank">oauth2client</a></b> is a client library for OAuth 2.0, which is used to access the users Google Earth Engine account.</li>
    <li> <b><a href="https://pypi.org/project/earthengine-api/" target="_blank">earthengine-api</a></b> allows developers to interact with Google Earth Engine using the Python programming language.</li>
    

In [ ]:
#Setup Environment
import os
import numpy as np
import urllib
import zipfile
import gdal
import glob
import tempfile
from tqdm.auto import tqdm

try:
    import googleapiclient
except:
    #!pip install google-api-python-client
    os.system('pip install google-api-python-client')

try:
    from oauth2client import crypt
except:
    #!pip install --upgrade oauth2client
    os.system('pip install --upgrade oauth2client')
    from oauth2client import crypt

try:
    import ee
except:
    #!pip install earthengine-api
    os.system('pip install earthengine-api')
    import ee


In [ ]:
#Login to Google Earth Engine
try:
    ee.Initialize()
except:
    print('In a terminal run: earthengine authenticate')

In [ ]:
#Define Constants and Functions

google_api_download_limit=33554432
def build_vrt(filename, input_file_list, targetAlignedPixels=True, separate=False, resampleAlg='near', resolution='highest'):
    vrt_options = gdal.BuildVRTOptions(resampleAlg=resampleAlg, resolution=resolution, separate=separate, targetAlignedPixels=targetAlignedPixels)
    ds=gdal.BuildVRT(filename,input_file_list,options=vrt_options)
    ds.FlushCache()

def download_tile_from_ge(W,E,S,N, collection_path='users/gena/global-hand/hand-100', prefix=None, fname=None, download_path=None, debug=False):
    if download_path is None:
        download_path = os.getcwd()
    if prefix is None:
        if fname is None:
            prefix='ge'            
    else: #prefix is not None
        if fname is not None: # and fname is not None
            print('When fname is specified prefix is ignored.')

    geom = ee.Geometry.Polygon( [[E, S], [W, S], [W, N], [E, N], [E, S]] )    
    try:  
        #GENA HAND dataset is an image collection
        hand = ee.ImageCollection(collection_path)
        hand_clip=ee.ImageCollection(hand.filterBounds(geom))
        hand_mosaic=hand_clip.reduce(ee.Reducer.mean());
        ge_path = hand_mosaic.getDownloadUrl({
            'scale': 30,
            'crs': 'EPSG:4326',
            'region': geom,
            'maxPixels': 1e10
        })        
    except:
        #This allows for future expansion to other datasets like NASADEM
        #e.g.
        #download_from_ge(W,E,S,N, collection_path="NASA/NASADEM_HGT/001",download_path='/home/jovyan/nasadem_test/ge_nasadem.vrt', debug=True, keep_downloads=False)
        hand = ee.Image(collection_path)
        ge_path = hand.getDownloadUrl({
            'scale': 30,
            'crs': 'EPSG:4326',
            'region': geom,
            'maxPixels': 1e10
        })

    if fname is None:
        fname=prefix+f'_{W}W_{E}E_{S}S_{N}N.zip'
    if debug: print(f'Downloading {fname} from: {ge_path}')
    output_path=os.path.join(download_path,fname)
    urllib.request.urlretrieve(ge_path, output_path)
    return output_path    

def download_from_ge(W,E,S,N, collection_path='users/gena/global-hand/hand-100', prefix=None, download_path=None, debug=False, keep_downloads=False):    
    #deal with input
    if prefix is None:
        prefix='ge'    
    if download_path is not None:
        download_folder=os.path.dirname(download_path)
        output_file=download_path
    else:
        download_folder=os.getcwd()
        output_file='ge_download.vrt'
    if not os.path.exists(download_folder):
        os.mkdir(download_folder)
    else:
        if not os.path.isdir(download_folder):
            print(f'Can not create download_folder: {download_folder}')
            raise ValueError 
    #start download
    estimated_size=estimate_size(W,E,S,N)
    tile_count=estimated_size/google_api_download_limit    
    if tile_count > 1: #32MB=32*1024*1024
        print(f'Area too large by a factor of: {tile_count}')        
    divider = np.int(np.ceil(np.sqrt(tile_count))+1)
    SN = np.linspace(S,N,divider)    
    WE = np.linspace(W,E,divider)
    if debug: print(f'SN: {SN}')
    if debug: print(f'WE: {WE}')
    s  = SN[0]
    w  = WE[0]
    zip_files = []
    print('Downloading...')
    for n in tqdm(SN[1:]):
        for e in tqdm(WE[1:]):
            if debug: print(f'w/e/s/n: {w}/{e}/{s}/{n}')
            #fname=prefix+f'_{w}W_{e}E_{s}S_{n}N.zip'
            fname='_'.join([prefix, bounding_box_to_string(w,e,s,n)])+'.zip'
            if debug: print(f'download: {os.path.join(download_folder,fname)}')
            if os.path.isfile(os.path.join(download_folder,fname)):
                zip_files.append(os.path.join(download_folder,fname))
            else:
                zip_files.append(download_tile_from_ge(w,e,s,n, collection_path=collection_path, fname=fname, download_path=download_folder, debug=debug))
            w=e.copy()
        s=n.copy()
        w=WE[0]
    if debug: print(zip_files)
    #start_splicing  
    print('Unzipping...')
    zip_contents=[]
    extract_folders=[os.path.splitext(f)[0] for f in zip_files]
    for f,extract_folder in zip(zip_files, extract_folders):        
        with zipfile.ZipFile(f, 'r') as zip_ref:  
            zip_contents.append(zip_ref.namelist())
            zip_ref.extractall(path=extract_folder)
    if debug: print(zip_contents)
    #convert zip_contents to list of full paths.
    vrt_contents=[ os.path.join(extract_folders[k],zip_contents[k][0]) for k in range(len(zip_files))]
    if debug: print(f'vrt_contents:{vrt_contents}')
    #combine with gdal
    build_vrt(output_file, vrt_contents, targetAlignedPixels=False) #targetAlignedPixels has to be False. Otherwise no file is created for some reason. 
    #cleanup
    if debug or keep_downloads:
        print('Skipping cleanup in debug mode or when keep_downloads is set.')
        print(f'Files NOT deleted: {zip_files}')        
    else:
        for f in zip_files:
            os.remove(f)
    print(f'Successfully generated:{output_file}')    
        
def estimate_size(W,E,S,N):
    pixels_per_deg=3601
    bytes_per_pixel=4
    return (E-W)*pixels_per_deg*(N-S)*pixels_per_deg*bytes_per_pixel

def coordinate_to_string(lat,lon, factor=4):
    """Format a pair of angles lat, lon as a string:
    string = coordinate_to_string(lat, lon)
    """
    def format_parts():
        fmt = '{prefix}{value:0>'+str(factor+3)+'d}'
        for angle, directions in zip((lon,lat), ['EW', 'NS']):
            if angle >= 0:
                yield fmt.format(prefix=directions[0], value=int(angle*10**factor))
            else:
                yield fmt.format(prefix=directions[1], value=int(-angle*10**factor))
    return '_'.join(format_parts())

def bounding_box_to_string(w,e,s,n, factor=3):
    """Format a pair of angles lat, lon as a string:
    string = coordinate_to_string(lat, lon)
    """
    s1=coordinate_to_string(n,w, factor=factor)
    s2=coordinate_to_string(s,e, factor=factor)
    return '_'.join([s1,s2])

In [ ]:
# Define Input Parameters

## 1. Bounding Box for the Area of Interest ##
W=92.0#-96.5001618 #upper left
N=25.0#41.5002284
E=93.0#-95.1003219 #lower right
S=24.0#39.9001804

## 2. Output Virtual Raster (VRT) file name ##
output_VRT='~/hand/ge_hand_Bangladesh_'+ bounding_box_to_string(W,E,S,N,factor=0) +'.vrt' 

In [ ]:
# Download and stitch tiles 
output_VRT=os.path.expanduser(output_VRT) #expand ~ to user home
if os.path.exists(output_VRT):
    if yesno(f"Overwrite file: {output_VRT}"):
        pass
    else:
        assert False

download_from_ge(W,E,S,N, download_path=output_VRT, debug=False, keep_downloads=False)

<font face="Calibri" size="2" color="gray"> <i> Version 0.1.0 - Batu Osmanoglu
    
<b>Change Log</b> <br>
2021/01/13: v0.1.0 <br>
-Initial version.<br>